In [2]:
# Disable GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
import numpy as np
import pandas as pd

# Load pretrained VGG16 model
model = tf.keras.applications.VGG16(weights='imagenet', include_top=True)

# Dict to store layer weights
conv_layer_weights = {}

# Loop over layers and extract weights from convolutional layers only
for layer in model.layers:
    if 'conv' in layer.name:
        weights = layer.get_weights()
        if weights:  # weights[0] = kernel, weights[1] = bias
            kernel = weights[0].flatten()  # Flatten to 1D
            conv_layer_weights[layer.name] = kernel

# Find max length to align all columns
max_len = max(len(w) for w in conv_layer_weights.values())

# Pad all arrays to same length
for layer in conv_layer_weights:
    weights = conv_layer_weights[layer]
    padded = np.pad(weights, (0, max_len - len(weights)), constant_values=np.nan)
    conv_layer_weights[layer] = padded

# Convert to DataFrame (each column = conv layer)
df = pd.DataFrame(conv_layer_weights)

# Save to CSV
output_path = "/kaggle/working/vgg16_conv_weights_by_layer.csv"
df.to_csv(output_path, index=False)

print(f"✅ Saved weights layer-by-layer to: '/kaggle/working/vgg16_conv_weights_by_layer.csv'")


2025-06-01 09:09:25.402869: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
✅ Saved weights layer-by-layer to: '/kaggle/working/vgg16_conv_weights_by_layer.csv'


In [3]:
import pandas as pd
import numpy as np

# Load the CSV file (adjust path as needed)
csv_path = '/kaggle/working/vgg16_conv_weights_by_layer.csv'
df = pd.read_csv(csv_path)

# Check for any non-numeric or NaN values
df = df.apply(pd.to_numeric, errors='coerce')  # Coerce non-numeric values to NaN
df = df.fillna(0)  # Replace NaNs with 0 (or use another strategy if needed)

# Get global min and max
min_val = df.min().min()
max_val = df.max().max()

# Handle case where min == max to avoid division by zero
if min_val == max_val:
    scale = 1
else:
    scale = 255 / (max_val - min_val)

# Vectorized conversion function
def float_to_int8_vectorized(x):
    return np.round((x - min_val) * scale - 128).astype(np.int8)

# Apply the function using vectorized approach
df_int8 = df.apply(float_to_int8_vectorized)

# Save the result
output_path = '/kaggle/working/VGG_Int8_Weights.csv'
df_int8.to_csv(output_path, index=False)

print(f"✅ Conversion completed and saved to {output_path}")


✅ Conversion completed and saved to /kaggle/working/VGG_Int8_Weights.csv
